In [1]:
import numpy as np
import pandas as pd
import catboost as cb

In [2]:
print(np.__version__)
print(pd.__version__)
print(cb.__version__)

1.19.2
1.2.2
0.24.1


In [3]:
train = pd.read_csv('public_train.csv')
test = pd.read_csv('private_input.csv')
submission = pd.read_csv('submission.csv')
well_details = pd.read_csv('well_details.csv')
new_depth = pd.read_csv('intake_depth v2.csv')

In [4]:
def get_base(train:pd.DataFrame, submission:pd.DataFrame) -> pd.DataFrame:
    """
    create a dataframe with submission.csv structure,
    features generated based on input data will be joined here later
    """
    
    train = (train
     .loc[~train.HR.isna() & ~train.Volumetric_Capacity.isna(),
          ['pump_id','Well_Number','Date','Volumetric_Capacity','HR']]
     .assign(is_test=0)
    )

    submission = (submission
     .assign(is_test=1))
    
    base = pd.concat([train, submission], ignore_index=True)
    base['HR'] = base['HR'].astype(int)
    base['Date'] = pd.to_datetime(base['Date'], format = '%Y-%m-%d')
    
    base = base.rename(columns={'pump_id':'pump_id_at_event'})
    
    return base


base = get_base(train, submission)
base.head()

,pump_id_at_event,Well_Number,Date,Volumetric_Capacity,HR,is_test
0,1.0,AGB001,2020-06-25,33.0,0,0
1,3.0,AGB002,2020-07-28,40.0,0,0
2,5.0,AGB003,2020-07-27,33.0,0,0
3,7.0,AGB004,2020-06-14,40.0,0,0
4,11.0,AGB006,2020-09-07,33.0,0,0


In [5]:
def get_all_data(train:pd.DataFrame, test:pd.DataFrame) -> pd.DataFrame:
    
    """
    merge all events from train and test wells,
    keep only those that either have labels or are present in test  
    """
    
    train = train.drop(columns = ['Volumetric_Capacity','HR'])
    
    all_data = pd.concat([train, test])

    all_data = all_data.loc[all_data['Well_Number']
                            .isin(base['Well_Number'].unique())
                           & ~all_data['PUMP_ID_PHASE'].isna()]

    all_data['unix_days'] = ((pd.to_datetime(all_data['Date'])
                             .astype(np.int64)/10**9/60/60/24)
                             .astype(np.int32))
    
    all_data['PUMP_ID_PHASE'] = all_data['PUMP_ID_PHASE'].astype(np.int32)

    all_data['Pump_Torque'] = np.nansum([all_data['Pump_Torque'],
                                         all_data['Pump_Torque.1']],
                                        axis = 0)

    all_data = all_data.drop(columns =['Pump_Torque.1', 'pump_id'])

    all_data = all_data.sort_values(by = 'Date', ignore_index = True)
    
    return all_data

all_data = get_all_data(train, test)
all_data

,Well_Number,Date,Gas_Rate,Water_Rate,Gas_Capacity,Water_Capacity,Gas_Pressure,Pump_Pressure,Pump_Speed,Pump_Torque,Pump_Volumetric_Eff,Downhole_Gauge_Fluid_Level_Above_Sensor,PUMP_ID_PHASE,unix_days
0,XEA001,2003-12-31,645.324610,0.398103,NaN,NaN,1020.42,537.79,200.0,0.0,89.268161,NaN,602,12417
1,XEA001,2004-01-01,683.315390,0.445959,NaN,NaN,896.32,544.69,200.0,0.0,99.998879,NaN,602,12418
2,XEA001,2004-01-02,710.511000,0.386656,NaN,NaN,868.74,565.37,200.0,0.0,86.701345,NaN,602,12419
3,XEA001,2004-01-03,695.875221,0.445959,NaN,NaN,875.63,572.26,200.0,0.0,99.998879,NaN,602,12420
4,XEA001,2004-01-04,716.635221,0.381887,NaN,NaN,896.32,544.69,200.0,0.0,85.631839,NaN,602,12421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595345,ZUP204,2021-01-27,0.000000,0.000000,815.868,0.099367,204.18,2485.22,0.0,0.0,0.000000,NaN,3736,18654
1595346,ZPP096,2021-01-27,1.043578,0.000074,685.080,0.018761,2.45,9.00,0.0,0.0,0.000000,240.186447,2097,18654
1595347,ZUO214,2021-01-27,0.000000,0.000222,1457.352,0.014627,7.90,2.60,0.0,0.0,0.000000,NaN,2911,18654
1595348,ZUP204,2021-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3736,18655


In [6]:
def duplicate(df):
    
    """
    multiply dataframes for each phase up to oldest data,
    simple example may be checked as follows:
    
    test_df = pd.DataFrame(
    {'Well_Number':['a','a','a','b','b','c','d','d','d','d'],
     'PUMP_ID_PHASE':[0,1,2,0,1,0,0,3,4,5],
     'unix_days':[0,1,2,3,4,5,6,7,8,9]}
    )

    duplicated_test_df = duplicate(test_df)

    duplicated_test_df"""
    
    clean = (
        df
        .groupby(['Well_Number','PUMP_ID_PHASE'])
        .Well_Number
        .count()
        .to_frame('tmp')
        .rename_axis(['Well_Number','phase_num_abs_true'])
    )
    
    phase_nums_info = (
        df
        .groupby(['Well_Number'])['PUMP_ID_PHASE']
        .agg([('min_phase_num_abs','min'),('max_phase_num_abs','max')])
    )
    
    df = (
        df
        .set_index('Well_Number')
        .join(phase_nums_info)
    )
    
    df['phase_num_rel'] = df['PUMP_ID_PHASE'] - df['min_phase_num_abs']
    df['phase_num_rel_max'] = df['max_phase_num_abs'] - df['min_phase_num_abs']
    
    large_df = pd.concat(
    [df.loc[(df.phase_num_rel<=i)&(df.phase_num_rel_max>=i)].assign(phase_num_rel_true = i) 
     for i in range(20)])
    
    large_df['phase_num_abs_true'] = (large_df['min_phase_num_abs']
                                      +large_df['phase_num_rel_true'])
    
    large_df = (large_df
                .set_index(['phase_num_abs_true'], append = True)
                .join(clean, how = 'inner')
    )
    
    days_info = (large_df
                 .groupby(large_df.index.names)
                 ['unix_days'].agg([('max_days','max')])
    )
    
    large_df = (large_df
                .join(days_info)
                .reset_index()
                .assign(days_to_event = lambda x: x.max_days-x.unix_days)
                .drop(['min_phase_num_abs',
                       'max_phase_num_abs',
                       'phase_num_rel',
                       'phase_num_rel_max',
                       'phase_num_rel_true',
                       'max_days',
                       'tmp'], axis = 1)
                .rename(columns={'PUMP_ID_PHASE':'pump_id','phase_num_abs_true':'pump_id_at_event'})  
    )
    
    return large_df

large_df = duplicate(all_data)
large_df

,Well_Number,pump_id_at_event,Date,Gas_Rate,Water_Rate,Gas_Capacity,Water_Capacity,Gas_Pressure,Pump_Pressure,Pump_Speed,Pump_Torque,Pump_Volumetric_Eff,Downhole_Gauge_Fluid_Level_Above_Sensor,pump_id,unix_days,days_to_event
0,AGB001,1,2019-05-12,0.000000,0.000000,3.114,0.016217,6.89,0.00,0.0,0.000000,0.0,NaN,1,18028,410
1,AGB001,1,2019-05-13,0.000000,0.006836,3.114,0.016217,1254.84,468.84,45.0,40.674600,0.0,NaN,1,18029,409
2,AGB001,1,2019-05-14,2.567618,0.008850,3.114,0.016217,1500.00,664.00,59.0,55.000083,0.0,NaN,1,18030,408
3,AGB001,1,2019-05-15,0.000000,0.008903,3.114,0.016217,1503.06,606.74,59.0,55.588620,0.0,NaN,1,18031,407
4,AGB001,1,2019-05-16,0.000000,0.008903,3.114,0.016217,1503.06,641.21,59.0,55.588620,0.0,NaN,1,18032,406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2796420,ZUP210,3751,2020-11-03,0.000000,0.000229,164.004,0.010811,73.56,69.58,0.0,0.000000,0.0,NaN,3751,18569,4
2796421,ZUP210,3751,2020-11-04,0.000000,0.000163,164.004,0.010811,1.59,1.03,0.0,0.000000,0.0,NaN,3751,18570,3
2796422,ZUP210,3751,2020-11-05,0.000000,0.000152,164.004,0.010811,1.68,1.14,0.0,0.000000,0.0,NaN,3751,18571,2
2796423,ZUP210,3751,2020-11-06,0.000000,0.000095,164.004,0.010811,0.94,0.46,0.0,0.000000,0.0,NaN,3751,18572,1


In [7]:
"""
read info on well details, create well groups based on well number
"""
well_details['well_group'] = well_details['Well_Number'].str[:3]
well_details = well_details.set_index('Well_Number')
well_details

,Completion_Type,Top_Perforation,Bottom_Perforation,Depth,Casing_Diam,well_group
Well_Number,,,,,,
AGB001,VERTICAL PRE-PERFORATED (SWELLABLE PACKERS),495.65,777.44,832.04,6.336,AGB
AGB002,VERTICAL FRAC,547.19,815.69,847.00,4.892,AGB
AGB003,VERTICAL FRAC,491.41,800.61,825.00,4.892,AGB
AGB004,VERTICAL FRAC,511.78,791.61,826.00,4.892,AGB
AGB005,VERTICAL FRAC,476.17,783.35,815.50,4.892,AGB
...,...,...,...,...,...,...
ZUP206,VERTICAL PRE-PERFORATED (SWELLABLE PACKERS),467.65,778.81,818.13,6.336,ZUP
ZUP207,VERTICAL PRE-PERFORATED (SWELLABLE PACKERS),541.69,821.40,855.00,6.336,ZUP
ZUP208,VERTICAL PRE-PERFORATED (SWELLABLE PACKERS),537.70,698.64,NaN,6.336,ZUP


In [8]:
"""
use input depth reading from previous phase
"""
new_depth = new_depth.drop_duplicates().drop([35])
new_depth['Tubing_Run_Date'] = pd.to_datetime(new_depth['Tubing_Run_Date'], format = '%d/%m/%Y')
new_depth['Tubing_Pull_Date'] = pd.to_datetime(new_depth['Tubing_Pull_Date'], format = '%d/%m/%Y')
new_depth['Date'] = new_depth.groupby('Well_Number')['Tubing_Run_Date'].shift(-1)
new_depth = (new_depth
             .loc[~new_depth['Date'].isna()]
             .set_index(['Well_Number','Date'])
             .drop(columns = ['Tubing_Run_Date','Tubing_Pull_Date']))

new_depth

Intake_Depth
Well_Number Date                    
AGB001      2020-06-25        803.30
AGB002      2020-07-28        793.14
AGB003      2020-07-27        770.10
AGB004      2020-06-14        755.46
AGB005      2020-07-05        758.31
...                              ...
ZUP207      2019-11-23        825.45
ZUP208      2019-10-30        701.52
            2020-07-04        705.01
ZUP209      2020-11-05        894.39
ZUP210      2020-11-07        901.39

[2508 rows x 1 columns]

In [9]:
"""
create simple time series statistic, for eg: mean, median, std, 5th and 95th quantiles
for different time intervals prior to expected event, for eg: 
past phase, past n days, past n days with on pump (speed > 0), past n days with off pump (speed == 0)
"""

current = (large_df
 .set_index(['Well_Number','pump_id_at_event'])
 .loc[lambda x: x['days_to_event']==0,['unix_days','Water_Capacity']]
 .rename(columns={'unix_days':'current_unix_days','Water_Capacity':'current_Water_Capacity'})
)


past_100 = (
    large_df
    .loc[large_df['days_to_event']<100]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_100_Water_Rate_mean = ('Water_Rate','mean'),
        past_100_Gas_Pressure_median = ('Gas_Pressure','median'),
        )
)


past_200 = (
    large_df
    .loc[large_df['days_to_event']<200]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_200_Water_Rate_mean=('Water_Rate','mean'),
        past_200_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_200_Pump_Speed_median=('Pump_Speed','median'),
        past_200_Pump_Speed_q95=('Pump_Speed',lambda x: x.quantile(.95)),
        )
)


past_300 = (
    large_df
    .loc[large_df['days_to_event']<300]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_300_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_300_Water_Capacity_std=('Water_Capacity','std'),
        past_300_Pump_Pressure_q95=('Pump_Pressure',lambda x: x.quantile(.95)),
        past_300_Pump_Torque_std=('Pump_Torque','std'),
        )
)


past_400 = (
    large_df
    .loc[large_df['days_to_event']<400]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_400_Gas_Rate_mean=('Gas_Rate','mean'),
        past_400_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_400_Pump_Pressure_mean=('Pump_Pressure','mean'),
        )
)


past_600 = (
    large_df
    .loc[large_df['days_to_event']<600]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_600_Gas_Rate_mean=('Gas_Rate','mean'),
        past_600_Water_Rate_median=('Water_Rate','median'),
        past_600_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_600_Pump_Speed_mean=('Pump_Speed','mean'),
        past_600_Pump_Volumetric_Eff_mean=('Pump_Volumetric_Eff','mean'),
        )
)


past_1000 = (
    large_df
    .loc[large_df['days_to_event']<1000]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_1000_Gas_Rate_mean=('Gas_Rate','mean'),
        past_1000_Water_Capacity_std=('Water_Capacity','std'),
        past_1000_Pump_Volumetric_Eff_mean=('Pump_Volumetric_Eff','mean'),
        past_1000_Pump_Volumetric_Eff_std=('Pump_Volumetric_Eff','std'),
        )
)


past_1200 = (
    large_df
    .loc[large_df['days_to_event']<1200]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_1200_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_1200_Pump_Pressure_mean=('Pump_Pressure','mean'),
        past_1200_Pump_Volumetric_Eff_mean=('Pump_Volumetric_Eff','mean'),
        past_1200_pump_id_nunique=('pump_id','nunique'),
        )
)


past_all = (
    large_df
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_all_Gas_Rate_mean=('Gas_Rate','mean'),
        past_all_Water_Rate_std=('Water_Rate','std'),
        past_all_Water_Rate_median=('Water_Rate', 'median'),
        past_all_Water_Rate_q95=('Water_Rate',lambda x: x.quantile(.95)),
        past_all_Pump_Volumetric_Eff_mean=('Pump_Volumetric_Eff','mean'),
        )
)


past_phase = (
    large_df
    .loc[large_df['pump_id']==large_df['pump_id_at_event']]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_phase_Gas_Pressure_median=('Gas_Pressure','median'),
        past_phase_Gas_Pressure_q05=('Gas_Pressure',lambda x: x.quantile(.05)),
        past_phase_Pump_Speed_median=('Pump_Speed','median'),
        past_phase_Pump_Volumetric_Eff_q95=('Pump_Volumetric_Eff',lambda x: x.quantile(.95)),
        )
)


past_phase_on = (
    large_df
    .loc[(large_df['pump_id']==large_df['pump_id_at_event'])
        &(large_df['Pump_Speed']>0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_phase_on_Gas_Rate_std=('Gas_Rate','std'),
        past_phase_on_Pump_Pressure_q95=('Pump_Pressure',lambda x: x.quantile(.95)),
        past_phase_on_Pump_Speed_median=('Pump_Speed','median'),
        past_phase_on_Pump_Volumetric_Eff_mean=('Pump_Volumetric_Eff','mean'),
        past_phase_on_Well_Number_count=('Well_Number','count'),
        )
)


past_phase_off = (
    large_df
    .loc[(large_df['pump_id']==large_df['pump_id_at_event'])
         &(large_df['Pump_Speed']==0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_phase_off_Gas_Rate_std=('Gas_Rate','std'),
        past_phase_off_Gas_Pressure_mean=('Gas_Pressure','mean'),
        past_phase_off_Gas_Pressure_q05=('Gas_Pressure',lambda x: x.quantile(.05)),
        past_phase_off_Pump_Pressure_median=('Pump_Pressure','median'),
        )
)


past_300_on = (
    large_df
    .loc[(large_df['days_to_event']<300)
         &(large_df['Pump_Speed']>0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_300_on_Water_Rate_std=('Water_Rate','std'),
        past_300_on_Water_Rate_median=('Water_Rate','median'),
        past_300_on_Water_Capacity_q95=('Water_Capacity',lambda x: x.quantile(.95)),
        past_300_on_Pump_Pressure_q95=('Pump_Pressure',lambda x: x.quantile(.95)),
        past_300_on_Pump_Speed_q05=('Pump_Speed',lambda x: x.quantile(.05)),
        )
)


past_300_off = (
    large_df
    .loc[(large_df['days_to_event']<300)
         &(large_df['Pump_Speed']==0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_300_off_Water_Rate_std=('Water_Rate','std'),
        past_300_off_Gas_Pressure_q05=('Gas_Pressure',lambda x: x.quantile(.05)),
        )
)


past_600_on = (
    large_df
    .loc[(large_df['days_to_event']<600)
         &(large_df['Pump_Speed']>0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_600_on_Water_Rate_std=('Water_Rate','std'),
        past_600_on_Water_Rate_median=('Water_Rate','median'),
        past_600_on_Gas_Capacity_q05=('Gas_Capacity',lambda x: x.quantile(.05)),
        past_600_on_Pump_Speed_q05=('Pump_Speed',lambda x: x.quantile(.05)),
        past_600_on_pump_id_nunique=('pump_id','nunique'),
        )
)


past_600_off = (
    large_df
    .loc[(large_df['days_to_event']<600)
         &(large_df['Pump_Speed']==0)]
    .groupby(['Well_Number','pump_id_at_event'])
    .agg(
        past_600_off_Gas_Rate_mean=('Gas_Rate','mean'),
        past_600_off_Gas_Rate_std=('Gas_Rate','std'),
        past_600_off_Water_Rate_mean=('Water_Rate','mean'),
        past_600_off_Water_Capacity_q95=('Water_Capacity',lambda x: x.quantile(.95)),
        )
)

In [10]:
"""
join all the reated features to a base dataframe, that was created in the begining
"""
df = (base
 .set_index(['Well_Number','Date'])
 .join(new_depth)
 .reset_index()
 .set_index('Well_Number')
 .join(well_details)
 .set_index('pump_id_at_event', append = True)
 .join(current)
 .join(past_100)
 .join(past_200)
 .join(past_300)
 .join(past_400)
 .join(past_600)
 .join(past_1000)
 .join(past_1200)
 .join(past_phase)
 .join(past_phase_on)
 .join(past_phase_off)
 .join(past_300_on)
 .join(past_300_off)
 .join(past_600_on)
 .join(past_600_off)
 .join(past_all)
 .reset_index()
)


In [11]:
"""feature selection is explained in detail in the other notebook"""

hr_columns = ['pump_id_at_event', 'Completion_Type', 'well_group',
'current_unix_days', 'past_100_Gas_Pressure_median',
'past_200_Water_Rate_mean', 'past_200_Pump_Speed_q95',
'past_300_Pump_Pressure_q95', 'past_600_Water_Rate_median',
'past_600_Pump_Speed_mean', 'past_600_Pump_Volumetric_Eff_mean',
'past_1000_Pump_Volumetric_Eff_mean',
'past_1200_Pump_Volumetric_Eff_mean', 'past_1200_pump_id_nunique',
'past_phase_Gas_Pressure_q05', 'past_phase_Pump_Speed_median',
'past_phase_Pump_Volumetric_Eff_q95', 'past_phase_on_Gas_Rate_std',
'past_phase_on_Pump_Pressure_q95', 'past_phase_on_Pump_Speed_median',
'past_phase_on_Pump_Volumetric_Eff_mean',
'past_phase_on_Well_Number_count', 'past_phase_off_Gas_Rate_std',
'past_phase_off_Gas_Pressure_mean', 'past_phase_off_Gas_Pressure_q05',
'past_phase_off_Pump_Pressure_median', 'past_300_on_Pump_Pressure_q95',
'past_300_off_Gas_Pressure_q05', 'past_600_on_Gas_Capacity_q05',
'past_600_on_pump_id_nunique', 'past_600_off_Water_Rate_mean',
'past_all_Pump_Volumetric_Eff_mean']


vc_columns = ['pump_id_at_event', 'Intake_Depth', 'Completion_Type', 'well_group',
'current_unix_days', 'current_Water_Capacity',
'past_100_Water_Rate_mean', 'past_200_Water_Rate_q95',
'past_200_Pump_Speed_median', 'past_300_Water_Rate_q95',
'past_300_Water_Capacity_std', 'past_300_Pump_Torque_std',
'past_400_Gas_Rate_mean', 'past_400_Water_Rate_q95',
'past_400_Pump_Pressure_mean', 'past_600_Gas_Rate_mean',
'past_600_Water_Rate_q95', 'past_600_Pump_Volumetric_Eff_mean',
'past_1000_Gas_Rate_mean', 'past_1000_Water_Capacity_std',
'past_1000_Pump_Volumetric_Eff_std', 'past_1200_Water_Rate_q95',
'past_1200_Pump_Pressure_mean', 'past_phase_Gas_Pressure_median',
'past_300_on_Water_Rate_std', 'past_300_on_Water_Rate_median',
'past_300_on_Water_Capacity_q95', 'past_300_on_Pump_Speed_q05',
'past_300_off_Water_Rate_std', 'past_600_on_Water_Rate_std',
'past_600_on_Water_Rate_median', 'past_600_on_Pump_Speed_q05',
'past_600_off_Gas_Rate_mean', 'past_600_off_Gas_Rate_std',
'past_600_off_Water_Capacity_q95', 'past_all_Gas_Rate_mean',
'past_all_Water_Rate_std', 'past_all_Water_Rate_median',
'past_all_Water_Rate_q95']

In [12]:
def cb_vc_test(random_state:int):
    
    cb_vc = cb.CatBoostRegressor(learning_rate = 0.02,
                              iterations = 3000,
                              loss_function = 'MAE',
                              task_type = 'CPU',
                              depth = 5,
                              random_state = random_state,
                             )
    cb_vc.fit(
        X = cb.Pool(df.loc[df.is_test==0,vc_columns],
                 df.loc[df.is_test==0,'Volumetric_Capacity'],
                 cat_features = ['Completion_Type', 'well_group']),
        silent = True
    )
    
    return cb_vc


def cb_hr_test(random_state:int):
    
    cb_hr = cb.CatBoostClassifier(learning_rate = 0.01,
                               auto_class_weights='Balanced',
                               iterations = 1000,
                               task_type = 'CPU',
                               boost_from_average = True,
                               random_state = random_state,
                              )

    cb_hr.fit(
        X = cb.Pool(df.loc[df.is_test==0,hr_columns],
                 df.loc[df.is_test==0,'HR'],
                 cat_features = ['Completion_Type', 'well_group']),
        silent = True
    )
    
    return cb_hr

In [13]:
"""create a blend from 3 catboost models trained with different seeds"""

n = 3

vc_models = []
hr_models = []

for i in range(n):
    vc_models.append(cb_vc_test(i))
    hr_models.append(cb_hr_test(i))
    
vc_merged = cb.sum_models(vc_models, weights = [1/n]*n)
hr_merged = cb.sum_models(hr_models, weights = [1/n]*n)

In [14]:
submission.Volumetric_Capacity = vc_merged.predict(df.loc[df.is_test==1,vc_columns])
submission.HR = hr_merged.predict(df.loc[df.is_test==1,hr_columns])>0

submission.to_csv('submission_final.csv', index = False)
submission

,pump_id,Well_Number,Date,Volumetric_Capacity,HR
0,9.0,AGB005,2020-07-05,33.419073,False
1,13.0,AGQ001,2017-12-10,22.292274,False
2,14.0,AGQ001,2019-06-17,24.580362,False
3,33.0,AGQ007,2017-05-07,31.871982,False
4,34.0,AGQ007,2020-05-09,24.255576,False
...,...,...,...,...,...
814,3720.0,ZUP197,2019-03-09,13.477193,False
815,3730.0,ZUP201,2018-10-08,15.657543,False
816,3743.0,ZUP207,2019-07-26,14.822069,False
817,3744.0,ZUP207,2019-11-23,15.635891,True
